In [7]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from multiprocessing.pool import ThreadPool

In [8]:
security_url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
stock_url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
corporate_url = "https://www.bseindia.com/corporates/corporate_act.aspx"
index_url = "https://www.bseindia.com/indices/IndexArchiveData.html"
bhav_copy_url = "https://www.bseindia.com/markets/MarketInfo/BhavCopy.aspx"
deliverables_url = "https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0"

In [18]:

def download_equity():

    if os.path.exists(os.path.join(os.getcwd(),"Data\Equity.csv")):
        print("Equity.csv exists")
        return

    path = os.path.join(os.getcwd(),"Data")
    print(path)

    chromeOptions = webdriver.ChromeOptions()
    # chromeOptions.add_argument("--headless")
    chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
    driver = webdriver.Chrome(ChromeDriverManager().install(),options = chromeOptions)
    driver.get(security_url)
    
    # to select Equity
    equity = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
    equity = Select(equity)
    equity.select_by_visible_text("Equity")  

    # to select Active
    active = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
    active = Select(active)
    active.select_by_visible_text("Active") 

    # to click submit 
    submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
    submit.send_keys(Keys.RETURN)

    # to download csv file
    driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i").click()
    time.sleep(3)
    driver.quit()

In [19]:
download_equity()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache
e:\IIITH\DataScience_2019501010\StockAnalysisTool\Data
 


In [1]:
path = os.path.join(os.getcwd(),"Data\Stock")
print(path)

def get_stocks(nums):

    # to set date
    def set_date(d,m,y):
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)
        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()
    
    def traverse_stock(security):
        
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        response = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[3]/div/div/table/tbody/tr[2]/td/div/div[2]/div/div/div[2]/div[1]/table/tbody/tr/td/div/div/div/ul/li")

        if response.text == "No Match Found":
            return None
        else:
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
            from_date.clear()
            from_date.click()
            set_date("15","Aug","2011")
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
            to_date.clear()
            to_date.click()
            set_date("21","Dec","2020")
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(1)
            driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[2]/div/div[1]/div[2]/span/a/i").click()
            time.sleep(3)

    if os.path.exists(os.path.join(os.getcwd(),"Data\Stock\\"+str(nums)+".csv")):
        return
    else:
        chromeOptions = webdriver.ChromeOptions()
        # chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        # url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
        driver.get(stock_url)
        traverse_stock(str(nums))
        driver.quit()

e:\IIITH\DataScience_2019501010\StockAnalysisTool\Data\Stock


In [38]:
equity = pd.read_csv(os.path.join(os.getcwd(),"Data\Equity.csv"))

In [39]:
values = equity["Security Code"].values
values[:10]

array([500002, 500003, 500008, 500009, 500010, 500012, 500013, 500014,
       500016, 500020], dtype=int64)

In [40]:
get_stocks(500020)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\VenkataSaiKrishna\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache
 


In [ ]:
# ThreadPool(4).map(get_stocks,values[:8])
for i in values[:8]:
    try: 
        get_stocks(i)
    except Exception as e:
        print(e)

In [ ]:
path = os.path.join(os.getcwd(),"Data\CorporateActions")
print(path)

def corporate_actions(nums):

    # to set date
    def get_date(d,m,y):
        month = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[1]')   
        for opt in month.find_elements_by_tag_name("option"):
            if opt.text == m:
                opt.click()
                break
        year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]')   
        for opt in year.find_elements_by_tag_name("option"):
            if opt.text == y:
                opt.click()
                break
        days = driver.find_element_by_xpath('/html/body/div[2]/table')
        for row in days.find_elements_by_tag_name('tr'):
            for col in row.find_elements_by_tag_name('td'):
                if col.text == d:
                    row.click()
                    return
    
    def traverse_actions(security):
        
        element = driver.find_element_by_xpath('')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        # response = driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[3]/div[3]/div[2]/div/table/tbody/tr/td/div/div/div/ul/li")

        # if response.text == "No Match Found":
        #     return None
        # else:
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
        from_date.clear()
        from_date.click()
        get_date("15","Aug","2015")
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
        to_date.clear()
        to_date.click()
        get_date("21","Dec","2020")
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i").click()

    if os.path.exists(os.path.join(os.getcwd(),"Data\CorporateActions\\"+str(nums)+".csv")):
        return
    else:
        chromeOptions = webdriver.ChromeOptions()
        # chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        # url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
        driver.get(corporate_url)
        traverse_actions(str(nums))
        time.sleep(3)
        driver.quit()
        os.rename(os.path.join(path,"Corporate_Actions.csv"),os.path.join(path,str(nums)+".csv"))


In [ ]:
# ThreadPool(1).map(corporate_actions,values[:1])
for i in values[:8]:
    try: 
        corporate_actions(i)
    except Exception as e:
        print(e)

In [2]:
path = os.path.join(os.getcwd(),"Data")
print(path)

def bse_index():

    # to set date
    def get_date(d,m,y):
        
        year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > "1991":
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)
        year.select_by_visible_text(year.options[0].text) 

        month = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    def traverse_index():
        indexes = driver.find_element_by_xpath('//*[@id="ddlIndex"]')
        indexes = Select(indexes)
        indexes.select_by_visible_text("S&P BSE 500")  

        from_date = driver.find_element_by_xpath('//*[@id="txtFromDt"]')
        from_date.clear()
        from_date.click()
        get_date("1","Aug","1991")
        to_date = driver.find_element_by_xpath('//*[@id="txtToDt"]')
        to_date.clear()
        to_date.click()
        get_date("21","Dec","2020")

        submit = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[5]/div/input')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/i").click()

    if os.path.exists(os.path.join(os.getcwd(),"Data\\"+"CSVForDate.csv")):
        return
    else:
        chromeOptions = webdriver.ChromeOptions()
        # chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        # url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
        driver.get(index_url)
        traverse_index()
        time.sleep(3)
        driver.quit()


e:\IIITH\DataScience_2019501010\StockAnalysisTool\Data


In [ ]:
bse_index()